In [1]:
###Mounting on drive
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
###Reading english and french files

with open("/content/drive/My Drive/fr-en.en", "r") as file:
  Eng_Fr_lang = file.read()
  
with open("/content/drive/My Drive/fr-en.fr", "r") as file:
  Fr_Eng_lang = file.read()

In [0]:
#Importing dependencies
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.tokenize import RegexpTokenizer
from string import punctuation

def strip_punctuation(s):
    return ''.join(c for c in s if c not in punctuation)

def preprocessing_word(file):
    file=file.lower()
    file_punc=''
    for string in file:
      for char in string:
        if char not in punctuation:
          file_punc += char
        else:
          file_punc += ''
    file_punc = file_punc.replace('\n',' ')
    return file_punc

##Preprocessing function  
def preprocessing(file):
    file=file.lower()
    file_punct = strip_punctuation(file)
    return file_punct

#Preprocessing the english and french corpus
Pre_Eng_Fr_Sent=preprocessing(Eng_Fr_lang)
Pre_Fr_Eng_Sent=preprocessing(Fr_Eng_lang)

In [0]:
###Splitting the data by sentence
Pre_Eng_Fr_Sent = Pre_Eng_Fr_Sent.split('\n')
Pre_Fr_Eng_Sent = Pre_Fr_Eng_Sent.split('\n')

In [22]:
###Checking the splitted data by slicing it
print(Pre_Eng_Fr_Sent[:5])
print(Pre_Fr_Eng_Sent[:5])

['resumption of the session', 'i declare resumed the session of the european parliament adjourned on friday 17 december 1999  and i would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period ', 'although  as you will have seen  the dreaded  millennium bug  failed to materialise  still the people in a number of countries suffered a series of natural disasters that truly were dreadful ', 'you have requested a debate on this subject in the course of the next few days  during this partsession ', 'in the meantime  i should like to observe a minute  s silence  as a number of members have requested  on behalf of all the victims concerned  particularly those of the terrible storms  in the various countries of the european union ']
['reprise de la session', 'je déclare reprise la session du parlement européen qui avait été interrompue le vendredi 17 décembre dernier et je vous renouvelle tous mes vux en espérant que vous avez passé de bonnes vacanc

In [0]:
###Splitting the data by words
eng_fr_words = []
for sentence in Pre_Eng_Fr_Sent:
  for word in sentence.split():
    eng_fr_words.append(word)

fr_eng_words = []
for sentence in Pre_Fr_Eng_Sent:
  for word in sentence.split():
    fr_eng_words.append(word)
  

In [0]:
####Declaring the vocabulary and taking the unique words in to count
Voc_english_french=set(eng_fr_words)
Unique_english_french = len(Voc_english_french)

Voc_french_english=set(fr_eng_words)
Unique_french_english = len(Voc_french_english)

In [0]:
##Initializing the probabilities of english in english vs french data
english_translation_probability = {}


  
for e in Voc_english_french:
  french_translation_probability = {}
  for f in Voc_french_english:
    french_translation_probability[f]=(1/Unique_english_french)
  english_translation_probability[e]=french_translation_probability

  
english_translation_probability_2 = {}


  
for e in Voc_english_german:
  german_translation_probability = {}
  for g in Voc_german_english:
    german_translation_probability[g]=(1/Unique_english_german)
  english_translation_probability_2[e]=german_translation_probability




In [9]:
english_translation_probability['1']['ce']

0.0002944640753828033

In [0]:
####main
import time 
from copy import deepcopy

## convergence point    
convergence_point = 0.0021  

##taking sentence pairing and initialisng sum as 0
def sentence_pair(counts,total_french,prob):
  for es,fs in zip(Pre_Eng_Fr_Sent, Pre_Fr_Eng_Sent):
    eng_word=es.split()
    fr_word=fs.split()
    sum = {}
    for ew in eng_word:
      sum[ew] = 0
      for fw in fr_word:
        sum[ew] += prob[ew][fw]

 ##setting count of aligned text probabilities.
    for ew in eng_word:
      for fw in fr_word:  
        counts[ew][fw] += prob[ew][fw]/sum[ew]                                       
        total_french[fw] += prob[ew][fw]/sum[ew]
  return (total_french, counts)

##updating the probabilities
def update_trans_prob(counts,total_french,prob):
  
  for fw in Voc_french_english:
    for ew in Voc_english_french:
      prob[ew][fw]=counts[ew][fw]/total_french[fw]
  return prob 

##calculating the error, separating the two corpus
def error_cal(prob,last_trans_prob):
  sum_of_difference_in_prob = 0
  for(ew,fw) in zip(Voc_english_french,Voc_french_english):
    sum_of_difference_in_prob += (prob[ew][fw] - last_trans_prob[ew][fw])**2
  sum_of_difference_in_prob = sum_of_difference_in_prob**0.5
  return sum_of_difference_in_prob  
  
## initialize count
def init_total_french():
  total_french = {}
  for fw in Voc_french_english:
    total_french[fw] = 0
  return total_french


def init_count_english_french():
  counts = {}
  for ew in Voc_english_french:
    init_foreign_lang = {}
    for fw in Voc_french_english:
      init_foreign_lang[fw] = 0
    counts[ew] = init_foreign_lang
  return counts

def word_translation_probabilities():
 
  convergence_point = 0.0021
  
## Initialize variables  
  error_value = 10
  prob = deepcopy(english_translation_probability)
  iter = 0
## loop for convergence  
  while (error_value > convergence_point):
        
    last_trans_prob = deepcopy(prob)
  
    total_french = init_total_french()
    
    counts = init_count_english_french()
    
    total_french, counts = sentence_pair(counts,total_french,prob)
                    
    prob = update_trans_prob(counts,total_french,prob)
    
    error_value = error_cal(prob,last_trans_prob) 
    
    iter  = iter + 1 
    print( iter , error_value)
  return prob      

In [12]:
##printing model per iteration and their respective error rates 
french_prob = word_translation_probabilities()

1 0.22944461939504426
2 0.09091737439599813
3 0.0559060489794136
4 0.04677681962671715
5 0.03706178796710107
6 0.028129306257189225
7 0.02187165003758181
8 0.01719731908898975
9 0.013533243401051206
10 0.01063970406962681
11 0.008363774797346037
12 0.006592815588191526
13 0.005235511264116988
14 0.004211728919917267
15 0.003448886639305327
16 0.002882074323934101
17 0.002456898238905609
18 0.002131978913173231
19 0.0018783067792840975


In [0]:
# prob

0.0

Printing word pairs with probability greater than 0.2. 

In [13]:
###priting english german translation probability tables
for ew in french_prob.keys():
  for fw,pb in french_prob[ew].items():   
    if pb > 0.2:
      print("{}           {}           {}".format(ew,fw,pb))

com           com           0.6296340671924926
next           administrative           0.23141968935235604
next           prochain           0.5351186123102892
next           appelle           0.5455296118479903
next           prochaines           0.27356009514526974
rules           réglementations           0.3777327095763659
rules           règlements           0.4968961703897571
rules           délais           0.34939571746727616
rules           règles           0.8937821684502523
interest           intérêt           0.5494196898050405
profits           gains           0.2032014565344922
profits           immédiats           0.2032014565344922
profits           court           0.2032014565344922
profits           privilégie           0.2032014565344922
exercise           exercice           0.715196467071886
important           important           0.8277543257177571
important           fonctionnement           0.27234875982255813
important           importante           0.8897146577

In [14]:
###french vs english corresponding sentences printed into tokens and separate lists for each sentence

for fs in Pre_Fr_Eng_Sent:
  ewds = []

  for fw in  fs.split():
    value_max = -999999
    for ew in french_prob.keys():
      ## getting maximum value code
        if french_prob[ew][fw] > value_max:
          value_max = french_prob[ew][fw]
          cewd = ew

    ewds.append(cewd)
  print(fs.split())
  print(ewds)

['reprise', 'de', 'la', 'session']
['session', 'of', 'the', 'partsession']
['je', 'déclare', 'reprise', 'la', 'session', 'du', 'parlement', 'européen', 'qui', 'avait', 'été', 'interrompue', 'le', 'vendredi', '17', 'décembre', 'dernier', 'et', 'je', 'vous', 'renouvelle', 'tous', 'mes', 'vux', 'en', 'espérant', 'que', 'vous', 'avez', 'passé', 'de', 'bonnes', 'vacances']
['i', 'festive', 'session', 'the', 'partsession', 'the', 'parliament', 'european', 'which', 'in', 'been', 'festive', 'the', 'friday', '17', 'december', 'like', 'and', 'i', 'you', 'festive', 'all', 'my', 'happy', 'in', 'festive', 'that', 'you', 'you', 'past', 'of', 'good', 'festive']
['comme', 'vous', 'avez', 'pu', 'le', 'constater', 'le', 'grand', 'bogue', 'de', 'l', 'an', '2000', 'ne', 's', 'est', 'pas', 'produit', 'en', 'revanche', 'les', 'citoyens', 'd', 'un', 'certain', 'nombre', 'de', 'nos', 'pays', 'ont', 'été', 'victimes', 'de', 'catastrophes', 'naturelles', 'qui', 'ont', 'vraiment', 'été', 'terribles']
['as', 'you